## Compare IWP, r_ice and t_ice for different ice particle shapes

In [1]:
import os
import netCDF4 as nc
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

Read results

In [2]:
path = '../data_TCWret/'
fname_spheroids = path + "spheroids.nc"
fname_spheres = path + 'spheres.nc'
fname_aggregates = path + 'aggregates.nc'
fname_bulletrosette = path + 'bulletrosettes.nc'
fname_droxtals = path + 'droxtals.nc'
fname_hollowcol = path + 'hollowcols.nc'
fname_plates = path + 'plates.nc'
fname_solidcol = path + 'solidcols.nc'

def read_tcwret(fname_tcwret):
    with nc.Dataset(fname_tcwret) as f:
        seconds = f.variables['time_of_measurement'][:]
        lwp = f.variables['liquid_water_path'][:]
        iwp = f.variables['ice_water_path'][:]
        lwp_err = f.variables['liquid_water_path_error'][:]
        iwp_err = f.variables['ice_water_path_error'][:]
        rliq = f.variables['liquid_water_effective_droplet_radius'][:]
        rice = f.variables['ice_water_effective_droplet_radius'][:]
        drliq = f.variables['liquid_water_effective_droplet_radius_error'][:]
        drice = f.variables['ice_water_effective_droplet_radius_error'][:]
        red_chi_2 = f.variables['reduced_chi_2'][:]
        t_cw = f.variables['liquid_water_optical_depth'][:] + f.variables['ice_water_optical_depth'][:]
        t_liq = f.variables['liquid_water_optical_depth'][:]
        t_ice = f.variables['ice_water_optical_depth'][:]
        dt_liq = f.variables['liquid_water_optical_depth_error'][:]
        dt_ice = f.variables['ice_water_optical_depth_error'][:]
        dt_cw = dt_liq + dt_ice
        
    idx_valid = np.where((red_chi_2 <= 1.0) & (t_cw <= 6.0))[0]
        
    return pd.DataFrame({'time': seconds[idx_valid], \
                         'ti(1)': t_ice[idx_valid], \
                         'dti(1)': dt_ice[idx_valid], \
                         'iwp(gm-2)': iwp[idx_valid], \
                         'diwp(gm-2)': iwp_err[idx_valid], \
                         'fi(1)': t_ice[idx_valid]/t_cw[idx_valid], \
                         'dfi(1)': np.abs(dt_ice[idx_valid]/t_cw[idx_valid]) + np.abs(dt_cw[idx_valid]*t_ice[idx_valid]/t_cw[idx_valid]**2), \
                         'rl(um)': rliq[idx_valid], \
                         'drl(um)': drliq[idx_valid], \
                         'ri(um)': rice[idx_valid], \
                         'dri(um)': drice[idx_valid]})

Calculate the intersection of more than two datasets by converting them into sets and reconverting to numpy arrays

In [3]:
def intersect(*arrays):
    sets = []
    for ii in arrays:
        sets.append(set(ii['time']))

    intersection_of_sets = sets[0]
    for ii in sets[1:]:
        intersection_of_sets = ii & intersection_of_sets
        
    return np.array(list(intersection_of_sets))

Compare ice effective radius

In [4]:
spheres = read_tcwret(fname_spheres)
spheroids = read_tcwret(fname_spheroids)
aggregates = read_tcwret(fname_aggregates)
bulletrosette = read_tcwret(fname_bulletrosette)
droxtals = read_tcwret(fname_droxtals)
hollowcol = read_tcwret(fname_hollowcol)
plates = read_tcwret(fname_plates)
solidcol = read_tcwret(fname_solidcol)

threshold = 0.1
idx_spheres = np.where(spheres['fi(1)'] > threshold)[0]
idx_aggregates = np.where(aggregates['fi(1)'] > threshold)[0]
idx_bulletrosette = np.where(bulletrosette['fi(1)'] > threshold)[0]
idx_droxtals = np.where(droxtals['fi(1)'] > threshold)[0]
idx_hollowcol = np.where(hollowcol['fi(1)'] > threshold)[0]
idx_plates = np.where(plates['fi(1)'] > threshold)[0]
idx_solidcol = np.where(solidcol['fi(1)'] > threshold)[0]
idx_spheroids = np.where(spheroids['fi(1)'] > threshold)[0]
spheres = spheres.iloc[idx_spheres]
aggregates = aggregates.iloc[idx_aggregates]
bulletrosette = bulletrosette.iloc[idx_bulletrosette]
droxtals = droxtals.iloc[idx_droxtals]
hollowcol = hollowcol.iloc[idx_hollowcol]
plates = plates.iloc[idx_plates]
solidcol = solidcol.iloc[idx_solidcol]
spheroids = spheroids.iloc[idx_spheroids]

times_intersect = intersect(spheres, spheroids, aggregates, bulletrosette, droxtals, hollowcol, plates, solidcol)
idx_spheres = np.intersect1d(spheres['time'], times_intersect, return_indices=True)[1]
idx_aggregates = np.intersect1d(aggregates['time'], times_intersect, return_indices=True)[1]
idx_bulletrosette = np.intersect1d(bulletrosette['time'], times_intersect, return_indices=True)[1]
idx_droxtals = np.intersect1d(droxtals['time'], times_intersect, return_indices=True)[1]
idx_hollowcol = np.intersect1d(hollowcol['time'], times_intersect, return_indices=True)[1]
idx_plates = np.intersect1d(plates['time'], times_intersect, return_indices=True)[1]
idx_solidcol = np.intersect1d(solidcol['time'], times_intersect, return_indices=True)[1]
idx_spheroids = np.intersect1d(spheroids['time'], times_intersect, return_indices=True)[1]

spheres = spheres.iloc[idx_spheres]
aggregates = aggregates.iloc[idx_aggregates]
bulletrosette = bulletrosette.iloc[idx_bulletrosette]
droxtals = droxtals.iloc[idx_droxtals]
hollowcol = hollowcol.iloc[idx_hollowcol]
plates = plates.iloc[idx_plates]
solidcol = solidcol.iloc[idx_solidcol]
spheroids = spheroids.iloc[idx_spheroids]

key = 'ri(um)'
a = np.array(spheres[key])
b = np.array(aggregates[key])
c = np.array(bulletrosette[key])
d = np.array(droxtals[key])
e = np.array(hollowcol[key])
f = np.array(plates[key])
g = np.array(solidcol[key])
h = np.array(spheroids[key])
key_err = 'dri(um)'
a_err = np.array(spheres[key_err])
b_err = np.array(aggregates[key_err])
c_err = np.array(bulletrosette[key_err])
d_err = np.array(droxtals[key_err])
e_err = np.array(hollowcol[key_err])
f_err = np.array(plates[key_err])
g_err = np.array(solidcol[key_err])
h_err = np.array(spheroids[key_err])
shapes_err = [a_err, b_err, c_err, d_err, e_err, f_err, g_err, h_err]
shapes = [a,b,c,d,e,f,g,h]
        
print(r"\begin{tabular}{|c|c|c|c|c|c|c|c|c|}")
print(r"\hline")
print(r"& Spheres & Aggregates & Bullet Rosettes & Droxtals & Hollow Column & Plates & Solid Columnds & Spheroids\\")
print(r"\hline")
labels = ["Spheres", "Aggregates", "Bullet Rosettes", "Droxtals", "Hollow Column", "Plates", "Solid Columns", "Spheroids"]
for ii in range(len(shapes)):
    std = labels[ii] + " &"
    for jj in range(len(shapes)):
        if np.std(shapes[ii]-shapes[jj]) > 9.68:
            std += ' $\SI{%.2f}{}**$ &' % (np.std(shapes[ii]-shapes[jj]))
        else:
            std += ' $\SI{%.2f}{}$ &' % (np.std(shapes[ii]-shapes[jj]))
    print(std + "\n" + r"\hline")

\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline
& Spheres & Aggregates & Bullet Rosettes & Droxtals & Hollow Column & Plates & Solid Columnds & Spheroids\\
\hline
Spheres & $\SI{0.00}{}$ & $\SI{6.76}{}$ & $\SI{14.15}{}**$ & $\SI{4.17}{}$ & $\SI{4.62}{}$ & $\SI{10.72}{}**$ & $\SI{3.88}{}$ & $\SI{3.18}{}$ &
\hline
Aggregates & $\SI{6.76}{}$ & $\SI{0.00}{}$ & $\SI{12.14}{}**$ & $\SI{6.76}{}$ & $\SI{5.60}{}$ & $\SI{9.50}{}$ & $\SI{6.58}{}$ & $\SI{6.86}{}$ &
\hline
Bullet Rosettes & $\SI{14.15}{}**$ & $\SI{12.14}{}**$ & $\SI{0.00}{}$ & $\SI{13.78}{}**$ & $\SI{12.89}{}**$ & $\SI{14.52}{}**$ & $\SI{13.45}{}**$ & $\SI{13.84}{}**$ &
\hline
Droxtals & $\SI{4.17}{}$ & $\SI{6.76}{}$ & $\SI{13.78}{}**$ & $\SI{0.00}{}$ & $\SI{4.78}{}$ & $\SI{10.88}{}**$ & $\SI{2.99}{}$ & $\SI{3.67}{}$ &
\hline
Hollow Column & $\SI{4.62}{}$ & $\SI{5.60}{}$ & $\SI{12.89}{}**$ & $\SI{4.78}{}$ & $\SI{0.00}{}$ & $\SI{10.19}{}**$ & $\SI{4.47}{}$ & $\SI{4.52}{}$ &
\hline
Plates & $\SI{10.72}{}**$ & $\SI{9.50}{}$ & $\SI{14.52}{

## Compare ice optical thickness

In [5]:
spheres = read_tcwret(fname_spheres)
spheroids = read_tcwret(fname_spheroids)
aggregates = read_tcwret(fname_aggregates)
bulletrosette = read_tcwret(fname_bulletrosette)
droxtals = read_tcwret(fname_droxtals)
hollowcol = read_tcwret(fname_hollowcol)
plates = read_tcwret(fname_plates)
solidcol = read_tcwret(fname_solidcol)

threshold = 0.0
idx_spheres = np.where(spheres['fi(1)'] > threshold)[0]
idx_aggregates = np.where(aggregates['fi(1)'] > threshold)[0]
idx_bulletrosette = np.where(bulletrosette['fi(1)'] > threshold)[0]
idx_droxtals = np.where(droxtals['fi(1)'] > threshold)[0]
idx_hollowcol = np.where(hollowcol['fi(1)'] > threshold)[0]
idx_plates = np.where(plates['fi(1)'] > threshold)[0]
idx_solidcol = np.where(solidcol['fi(1)'] > threshold)[0]
idx_spheroids = np.where(spheroids['fi(1)'] > threshold)[0]
spheres = spheres.iloc[idx_spheres]
aggregates = aggregates.iloc[idx_aggregates]
bulletrosette = bulletrosette.iloc[idx_bulletrosette]
droxtals = droxtals.iloc[idx_droxtals]
hollowcol = hollowcol.iloc[idx_hollowcol]
plates = plates.iloc[idx_plates]
solidcol = solidcol.iloc[idx_solidcol]
spheroids = spheroids.iloc[idx_spheroids]

times_intersect = intersect(spheres, spheroids, aggregates, bulletrosette, droxtals, hollowcol, plates, solidcol)
idx_spheres = np.intersect1d(spheres['time'], times_intersect, return_indices=True)[1]
idx_aggregates = np.intersect1d(aggregates['time'], times_intersect, return_indices=True)[1]
idx_bulletrosette = np.intersect1d(bulletrosette['time'], times_intersect, return_indices=True)[1]
idx_droxtals = np.intersect1d(droxtals['time'], times_intersect, return_indices=True)[1]
idx_hollowcol = np.intersect1d(hollowcol['time'], times_intersect, return_indices=True)[1]
idx_plates = np.intersect1d(plates['time'], times_intersect, return_indices=True)[1]
idx_solidcol = np.intersect1d(solidcol['time'], times_intersect, return_indices=True)[1]
idx_spheroids = np.intersect1d(spheroids['time'], times_intersect, return_indices=True)[1]

spheres = spheres.iloc[idx_spheres]
aggregates = aggregates.iloc[idx_aggregates]
bulletrosette = bulletrosette.iloc[idx_bulletrosette]
droxtals = droxtals.iloc[idx_droxtals]
hollowcol = hollowcol.iloc[idx_hollowcol]
plates = plates.iloc[idx_plates]
solidcol = solidcol.iloc[idx_solidcol]
spheroids = spheroids.iloc[idx_spheroids]


key = 'ti(1)'
a = np.array(spheres[key])
b = np.array(aggregates[key])
c = np.array(bulletrosette[key])
d = np.array(droxtals[key])
e = np.array(hollowcol[key])
f = np.array(plates[key])
g = np.array(solidcol[key])
h = np.array(spheroids[key])
key_err = 'dti(1)'
a_err = np.array(spheres[key_err])
b_err = np.array(aggregates[key_err])
c_err = np.array(bulletrosette[key_err])
d_err = np.array(droxtals[key_err])
e_err = np.array(hollowcol[key_err])
f_err = np.array(plates[key_err])
g_err = np.array(solidcol[key_err])
h_err = np.array(spheroids[key_err])
shapes_err = [a_err, b_err, c_err, d_err, e_err, f_err, g_err, h_err]
shapes = [a,b,c,d,e,f,g,h]

print(r"\begin{tabular}{|c|c|c|c|c|c|c|c|c|}")
print(r"\hline")
print(r"& SPH & A & BR & D & HC & P & SC & SPO\\")
print(r"\hline")
labels = ["SPH", "A", "BR", "D", "HC", "P", "SC", "SPO"]
for ii in range(len(shapes)):
    std = labels[ii] + " &"
    for jj in range(len(shapes)):
        if np.std(shapes[ii]-shapes[jj]) > 0.56:
            std += ' $\SI{%.2f}{}**$ &' % (np.std(shapes[ii]-shapes[jj]))
        else:
            std += ' $\SI{%.2f}{}$ &' % (np.std(shapes[ii]-shapes[jj]))
    print(std + "\n" + r"\hline")

\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline
& SPH & A & BR & D & HC & P & SC & SPO\\
\hline
SPH & $\SI{0.00}{}$ & $\SI{0.49}{}$ & $\SI{0.57}{}**$ & $\SI{0.39}{}$ & $\SI{0.38}{}$ & $\SI{0.49}{}$ & $\SI{0.36}{}$ & $\SI{0.30}{}$ &
\hline
A & $\SI{0.49}{}$ & $\SI{0.00}{}$ & $\SI{0.39}{}$ & $\SI{0.43}{}$ & $\SI{0.41}{}$ & $\SI{0.36}{}$ & $\SI{0.41}{}$ & $\SI{0.47}{}$ &
\hline
BR & $\SI{0.57}{}**$ & $\SI{0.39}{}$ & $\SI{0.00}{}$ & $\SI{0.51}{}$ & $\SI{0.47}{}$ & $\SI{0.37}{}$ & $\SI{0.52}{}$ & $\SI{0.55}{}$ &
\hline
D & $\SI{0.39}{}$ & $\SI{0.43}{}$ & $\SI{0.51}{}$ & $\SI{0.00}{}$ & $\SI{0.41}{}$ & $\SI{0.46}{}$ & $\SI{0.37}{}$ & $\SI{0.40}{}$ &
\hline
HC & $\SI{0.38}{}$ & $\SI{0.41}{}$ & $\SI{0.47}{}$ & $\SI{0.41}{}$ & $\SI{0.00}{}$ & $\SI{0.42}{}$ & $\SI{0.37}{}$ & $\SI{0.36}{}$ &
\hline
P & $\SI{0.49}{}$ & $\SI{0.36}{}$ & $\SI{0.37}{}$ & $\SI{0.46}{}$ & $\SI{0.42}{}$ & $\SI{0.00}{}$ & $\SI{0.41}{}$ & $\SI{0.48}{}$ &
\hline
SC & $\SI{0.36}{}$ & $\SI{0.41}{}$ & $\SI{0.52}{}$ & $\SI{0.37}{}$

Compare ice water path

In [6]:
spheres = read_tcwret(fname_spheres)
spheroids = read_tcwret(fname_spheroids)
aggregates = read_tcwret(fname_aggregates)
bulletrosette = read_tcwret(fname_bulletrosette)
droxtals = read_tcwret(fname_droxtals)
hollowcol = read_tcwret(fname_hollowcol)
plates = read_tcwret(fname_plates)
solidcol = read_tcwret(fname_solidcol)

threshold = 0.0

idx_spheres = np.where(spheres['fi(1)'] > threshold)[0]
idx_aggregates = np.where(aggregates['fi(1)'] > threshold)[0]
idx_bulletrosette = np.where(bulletrosette['fi(1)'] > threshold)[0]
idx_droxtals = np.where(droxtals['fi(1)'] > threshold)[0]
idx_hollowcol = np.where(hollowcol['fi(1)'] > threshold)[0]
idx_plates = np.where(plates['fi(1)'] > threshold)[0]
idx_solidcol = np.where(solidcol['fi(1)'] > threshold)[0]
idx_spheroids = np.where(spheroids['fi(1)'] > threshold)[0]

spheres = spheres.iloc[idx_spheres]
aggregates = aggregates.iloc[idx_aggregates]
bulletrosette = bulletrosette.iloc[idx_bulletrosette]
droxtals = droxtals.iloc[idx_droxtals]
hollowcol = hollowcol.iloc[idx_hollowcol]
plates = plates.iloc[idx_plates]
solidcol = solidcol.iloc[idx_solidcol]
spheroids = spheroids.iloc[idx_spheroids]

times_intersect = intersect(spheres, spheroids, aggregates, bulletrosette, droxtals, hollowcol, plates, solidcol)
idx_spheres = np.intersect1d(spheres['time'], times_intersect, return_indices=True)[1]
idx_aggregates = np.intersect1d(aggregates['time'], times_intersect, return_indices=True)[1]
idx_bulletrosette = np.intersect1d(bulletrosette['time'], times_intersect, return_indices=True)[1]
idx_droxtals = np.intersect1d(droxtals['time'], times_intersect, return_indices=True)[1]
idx_hollowcol = np.intersect1d(hollowcol['time'], times_intersect, return_indices=True)[1]
idx_plates = np.intersect1d(plates['time'], times_intersect, return_indices=True)[1]
idx_solidcol = np.intersect1d(solidcol['time'], times_intersect, return_indices=True)[1]
idx_spheroids = np.intersect1d(spheroids['time'], times_intersect, return_indices=True)[1]

spheres = spheres.iloc[idx_spheres]
aggregates = aggregates.iloc[idx_aggregates]
bulletrosette = bulletrosette.iloc[idx_bulletrosette]
droxtals = droxtals.iloc[idx_droxtals]
hollowcol = hollowcol.iloc[idx_hollowcol]
plates = plates.iloc[idx_plates]
solidcol = solidcol.iloc[idx_solidcol]
spheroids = spheroids.iloc[idx_spheroids]

key = 'iwp(gm-2)'
a = np.array(spheres[key])
b = np.array(aggregates[key])
c = np.array(bulletrosette[key])
d = np.array(droxtals[key])
e = np.array(hollowcol[key])
f = np.array(plates[key])
g = np.array(solidcol[key])
h = np.array(spheroids[key])

key_err = 'diwp(gm-2)'
a_err = np.array(spheres[key_err])
b_err = np.array(aggregates[key_err])
c_err = np.array(bulletrosette[key_err])
d_err = np.array(droxtals[key_err])
e_err = np.array(hollowcol[key_err])
f_err = np.array(plates[key_err])
g_err = np.array(solidcol[key_err])
h_err = np.array(spheroids[key_err])
shapes_err = [a_err, b_err, c_err, d_err, e_err, f_err, g_err, h_err]
shapes = [a,b,c,d,e,f,g,h]

print(r"\begin{tabular}{|c|c|c|c|c|c|c|c|c|}")
print(r"\hline")
print(r"& SPH & A & BR & D & HC & P & SC & SPO\\")
print(r"\hline")
labels = ["SPH", "A", "BR", "D", "HC", "P", "SC", "SPO"]
for ii in range(len(shapes)):
    std = labels[ii] + " &"
    for jj in range(len(shapes)):
        if np.std(shapes[ii]-shapes[jj]) > 9.85:
            std = ' $\SI{%.2f}{}**$ &' % (np.std(shapes[ii]-shapes[jj])) 
        else:
            std += ' $\SI{%.2f}{}$ &' % (np.std(shapes[ii]-shapes[jj]))
    print(std + "\n" + r"\hline")

\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline
& SPH & A & BR & D & HC & P & SC & SPO\\
\hline
SPH & $\SI{0.00}{}$ & $\SI{7.58}{}$ & $\SI{9.31}{}$ & $\SI{4.37}{}$ & $\SI{6.00}{}$ & $\SI{7.80}{}$ & $\SI{3.90}{}$ & $\SI{3.39}{}$ &
\hline
A & $\SI{7.58}{}$ & $\SI{0.00}{}$ & $\SI{8.32}{}$ & $\SI{7.08}{}$ & $\SI{6.80}{}$ & $\SI{7.29}{}$ & $\SI{7.04}{}$ & $\SI{7.79}{}$ &
\hline
BR & $\SI{9.31}{}$ & $\SI{8.32}{}$ & $\SI{0.00}{}$ & $\SI{9.13}{}$ & $\SI{8.85}{}$ & $\SI{9.13}{}$ & $\SI{9.08}{}$ & $\SI{9.72}{}$ &
\hline
D & $\SI{4.37}{}$ & $\SI{7.08}{}$ & $\SI{9.13}{}$ & $\SI{0.00}{}$ & $\SI{6.49}{}$ & $\SI{7.37}{}$ & $\SI{4.20}{}$ & $\SI{4.93}{}$ &
\hline
HC & $\SI{6.00}{}$ & $\SI{6.80}{}$ & $\SI{8.85}{}$ & $\SI{6.49}{}$ & $\SI{0.00}{}$ & $\SI{8.04}{}$ & $\SI{6.27}{}$ & $\SI{5.71}{}$ &
\hline
P & $\SI{7.80}{}$ & $\SI{7.29}{}$ & $\SI{9.13}{}$ & $\SI{7.37}{}$ & $\SI{8.04}{}$ & $\SI{0.00}{}$ & $\SI{6.99}{}$ & $\SI{8.45}{}$ &
\hline
SC & $\SI{3.90}{}$ & $\SI{7.04}{}$ & $\SI{9.08}{}$ & $\SI{4.20}{}$ & $